In [4]:

%reload_ext autoreload
from pathlib import Path
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader
import numpy as np
# import matplotlib.pyplot as plt
import random
from einops import rearrange


In [5]:

%autoreload 2
from crossView_UCLA_ske import NUCLA_CrossView
from Dyan import Dyan
# from torch.utils.tensorboard import SummaryWriter

In [6]:
setup = 'setup1'
dataType = '2D'
sampling = 'Single'
nClip = 1
maskType = 'None'
nw = 8

learning_rate = 1.0e-4

In [7]:
seed = 1337

# Set seed for PyTorch (CPU)
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# Set seed for PyTorch (GPU)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If you use multiple GPUs

# Make PyTorch operations deterministic (slows down computation but improves reproducibility)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
print(f'Pytorch Version: {torch.__version__} \n {torch.cuda.get_device_name()}')

test_name = 'default'
output_path = Path('/home/rsl/sparse_recovery_tests/tests').joinpath(test_name)
output_path.mkdir(parents=True, exist_ok=True)

data_files_list = Path().home().joinpath('data', 'N-UCLA_MA_3D')
if data_files_list.exists():
        data_files_list = data_files_list.as_posix() + f"/data/CV/{setup}/"
else:
    raise FileExistsError


Pytorch Version: 2.5.1 
 NVIDIA GeForce RTX 2080 Ti


In [8]:

trainSet = NUCLA_CrossView(root_list=data_files_list, phase='train',
                            setup=setup, dataType=dataType,
                            sampling=sampling, nClip=nClip,
                            T=36, maskType=maskType) 
trainLoader = DataLoader(trainSet, shuffle=True,
                             batch_size=16, num_workers=nw)

testSet = NUCLA_CrossView(root_list=data_files_list, phase='test',
                            setup=setup, dataType=dataType,
                            sampling=sampling, nClip=nClip,
                            T=36, maskType=maskType) 
testLoader = DataLoader(trainSet, shuffle=False,
                             batch_size=16, num_workers=nw)

In [7]:
train_views = [sample[0] for sample in trainSet.samples_list]
print(np.unique(train_views, return_counts=True))
    

(array(['view_1', 'view_2'], dtype='<U6'), array([516, 504]))


In [8]:
s_min = []
s_max = []
for i, sample in enumerate(trainLoader):
    # setup batches
    skeletons = sample['input_skeletons']['normSkeleton'].float().to(device)
    # t = skeletons.shape[1] # (batch_size x num_clips) x t x num_joint x dim_joint
    # gt_skeletons = skeletons.reshape(skeletons.shape[0], t, -1).to(device)
    gt_skeletons = rearrange(skeletons, 'b u f j c -> (b u) f j c').cpu()
    # if i == 0:
    #     break


    for sample in range(gt_skeletons.shape[0]):
        for frame in range(gt_skeletons.shape[1]):
            ske = gt_skeletons[sample][frame]
            s_min.append(ske.min())
            s_max.append(ske.max())

print(np.unique(s_min, return_counts=True))
print(np.unique(s_max, return_counts=True))

(array([0.], dtype=float32), array([36720]))
(array([1.], dtype=float32), array([36720]))


In [25]:
gt_skeletons[0][0]

tensor([[2.3931e-01, 1.1672e-02],
        [4.9270e-01, 1.5598e-01],
        [2.5364e-01, 1.5220e-01],
        [2.3923e-01, 2.7375e-01],
        [0.0000e+00, 3.1163e-01],
        [7.3249e-01, 1.5967e-01],
        [1.0000e+00, 3.1184e-01],
        [4.0847e-01, 3.4977e-01],
        [4.5062e-01, 4.8289e-01],
        [2.6785e-01, 4.7530e-01],
        [3.6659e-01, 7.1118e-01],
        [3.6555e-01, 9.0867e-01],
        [5.9145e-01, 4.9040e-01],
        [5.9120e-01, 7.2998e-01],
        [6.0565e-01, 9.4289e-01],
        [2.3868e-01, 0.0000e+00],
        [3.5122e-01, 5.2434e-05],
        [0.0000e+00, 0.0000e+00],
        [5.9100e-01, 1.1699e-02],
        [3.5248e-01, 9.8864e-01],
        [4.6457e-01, 1.0000e+00],
        [6.9030e-01, 9.5812e-01],
        [1.4077e-01, 9.4667e-01],
        [1.4074e-01, 9.2778e-01],
        [3.8040e-01, 9.2414e-01]], device='cuda:0')

In [10]:
training_loss = []
C_list = []
dyan = Dyan(device, config_filename='dyan_config', train_new_dict=True)
dyan.to(device)
dyan.eval()
mseloss = nn.MSELoss()

optimizer = torch.optim.SGD(filter(lambda x: x.requires_grad, dyan.parameters()), 
                            lr=learning_rate, weight_decay=0.001, momentum=0.9)

for i, sample in enumerate(trainLoader):
    # setup batches
    skeletons = sample['input_skeletons']['normSkeleton'].float().to(device)
    t = skeletons.shape[1] # (batch_size x num_clips) x t x num_joint x dim_joint
    gt_skeletons = skeletons.reshape(skeletons.shape[0], t, -1).to(device)
    
    
    pred_skeletons, C_pred = dyan(gt_skeletons)
    training_loss.append(mseloss(pred_skeletons, gt_skeletons))
    C_list.append(C_pred)
    

Required key 'dictionary_filename' not found in configuration file. Using default value of None


AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [ ]:
dyan.to(device)
dyan.dictionary_.device

device(type='cpu')

In [5]:
def gridRing(N):
    # epsilon_low = 0.25
    # epsilon_high = 0.15
    # rmin = (1 - epsilon_low)
    # rmax = (1 + epsilon_high)

    # epsilon_low = 0.25
    epsilon_low = 0.15
    epsilon_high = 0.15
    rmin = (1 - epsilon_low)
    rmax = (1 + epsilon_high)

    thetaMin = 0.001
    thetaMax = np.pi - 0.001
    delta = 0.001
    # Npole = int(N / 4)
    Npole = int(N/2)
    Pool = generateGridPoles(delta, rmin, rmax, thetaMin, thetaMax)
    M = len(Pool)

    idx = random.sample(range(0, M), Npole)
    P = Pool[idx]
    # Pall = np.concatenate((P, -P, np.conjugate(P), np.conjugate(-P)), axis=0)
    Pall = np.concatenate((P, np.conjugate(P)), axis=0)  # mirror once

    return P, Pall


## Generate the grid on poles
def generateGridPoles(delta, rmin, rmax, thetaMin, thetaMax):
    rmin2 = pow(rmin, 2)
    rmax2 = pow(rmax, 2)
    xv = np.arange(-rmax, rmax, delta)
    x, y = np.meshgrid(xv, xv, sparse=False)
    mask = np.logical_and(np.logical_and(x ** 2 + y ** 2 >= rmin2, x ** 2 + y ** 2 <= rmax2),
                          np.logical_and(np.angle(x + 1j * y) >= thetaMin, np.angle(x + 1j * y) <= thetaMax))
    px = x[mask]
    py = y[mask]
    P = px + 1j * py

    return P


def get_Drr_Dtheta(N):
    P, Pall = gridRing(N)
    Drr = abs(P)
    # Drr = torch.from_numpy(Drr).float()
    Dtheta = np.angle(P)
    # Dtheta = torch.from_numpy(Dtheta).float()s
    return Drr, Dtheta

In [17]:
P, Pall = gridRing(161)
Drr = abs(P)
# Drr = torch.from_numpy(Drr).float()
Dtheta = np.angle(P)
# Dtheta = torch.from_numpy(Dtheta).float()s
print(P.shape)
print(Pall.shape)

(80,)
(160,)


In [ ]:
import torch

# some constants
n_poles = (161 - 1) / 2
epsilon = torch.tensor(0.15).double()
delta = torch.tensor(0.001).double()

# define the domain
r2_min = torch.pow(1 - epsilon, 2)
r2_max = torch.pow(1 + epsilon, 2)
theta_min = delta
theta_max = torch.pi - delta


mesh_range = torch.arange(-r2_max, r2_max, delta)

torch.meshgrid(mesh_range, mesh_range, sparse=False)


In [9]:
np.arange(10) % 1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])